In [197]:
#ЧБ изображения 6х6, где проверяется какой знак написан "+ - = ! х /" 6 знаков

#36 на входе (пиксели 00 - ij (развернутые по строкам)), 6 на выходе (получившийся знак)

In [198]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from ipywidgets import Button, GridspecLayout, VBox, Layout
import random

In [199]:
#Число файлов с каждым знаком
VIBORKA = 15
#Скорость обучения
SPEED = 0.2
#Допустимая ошибка
#EPSILON = 0.1
#Вектор порогов
VECTOR_POROGOV = []
#Ограничение эопх
EPOCH = 200
test_img_path = 'Input.png'
for i in range(6):
    VECTOR_POROGOV.append(random.random()-0.5)
# Список знаков
symbols = ['+', '-', '=', '!', 'x', 'l']
symbols_index = [0, 1, 2, 3, 4, 5]
# '+' - 0, '-' - 1, '=' - 2, '!' - 3, 'x' - 4, 'l' - 5
Par = {0:'+', 1:'-', 2:'=', 3:'!', 4:'x', 5:'l'}
# Размер изображения
img_size = (6, 6)


In [200]:
#Получение пикселей из изображения
def pix(path):
    "path - ссылка на изображение"
    img = Image.open(path)
    img = img.convert("L") #Конверт в один канал
    pixels = img.load()
    return pixels

In [201]:
# Преобразование изображения в вектор
def img_to_vector(pixels):
    "pixels - pix(path)"
    vector = []
    for i in range(6):
        for j in range(6):
            if pixels[i,j] > (255/2):
               vector.append(0)
            else:
               vector.append(1)
    vector.append(1)
    return vector

In [202]:
#Генерация данных от обучающей выборки
def gen_vibor():
    X = []
    Y = []
    path = ""
    for i in range(1,VIBORKA+1):
        for j in range(6):
            y = []
            path = f"Study/{Par[j]}_{i}.png"
            X.append(img_to_vector(pix(path)))
            for l in range(j):
                y.append(0)
            y.append(1)
            for l in range(5-j):
                y.append(0)
            Y.append(y)
    return X, Y

In [203]:
#Инициализация весов
def inec_W():
    "36*6 в W1 и 6 в W2 (0-1)"
    W1 = []
    for i in range(36):
        if i!= 0:
            W1.append(w1)
        w1 = []
        for j in range(6):
            w1.append(random.random()-0.5)
    W1.append(w1)
    W1.append(VECTOR_POROGOV)
    W1 = np.array(W1)
    return W1

In [204]:
def act(y):
    if y > 0.7:
        return int(1)
    else:
        return int(0)

In [205]:
#Вектор ошибок
def error(real, predicted):
    "Список реальных значений, список предсказанных"
    Error_vec = []
    for i in range(min(len(real),len(predicted))):
        Error_vec.append(real[i] - predicted[i])
    return Error_vec

In [206]:
#Транспонирование
def trans(masiv):  
     result = []
     for i in range(len(masiv)):
               result.append([masiv[i]])
     return result

In [207]:
#Задание массива
def mas(masiv):
    result=[]
    for i in range(len(masiv)):
        result.append(masiv[i])
    return [result]

In [208]:
#Корректировка весов
def correct(real, errors, W):
    D = SPEED * np.dot(trans(real),mas(errors))
    W = np.add(W, D)
    return W

In [209]:
def predict(path, W):
    X = img_to_vector(pix(path))
    Y = np.dot(X, W)
    output = None
    max = -100
    for i in range(len(Y)):
        if Y[i]>max:
            output = i
            max = Y[i]
        Y[i] = act(Y[i])
    return Par[output]

In [210]:
#Выполнение пока ошибка уменьшается и больше допустимой
epoch = 0
W = inec_W()
X, Y = gen_vibor()
X = np.array(X)
Y = np.array(Y) 
while epoch < EPOCH:
    epoch+=1       
    Ypredict = np.dot(X, W)
    for i in range(len(Ypredict)):
        for j in range(len(Ypredict[0])):
            Ypredict[i][j] = act(Ypredict[i][j])
    for i in range(VIBORKA*6):
        Ei = error(Y[i], Ypredict[i])
        W = correct(X[i], Ei, W)
        #print(W)

In [211]:
# Размер изображения
img_size = (6, 6)

# Инициализация пустого изображения с белыми пикселями
canvas = np.ones(img_size, dtype=np.uint8)

# Создаем начальную сетку кнопок для рисования
grid = GridspecLayout(img_size[0], img_size[1], width='300px', height='300px')

def update_pixel(x, y, button):
    canvas[y, x] = 1 - canvas[y, x]  # Инвертируем пиксель
    button.style.button_color = 'white' if canvas[y, x] == 1 else 'black'  # Обновляем цвет кнопки

# Заполняем сетку кнопками и добавляем их на экран
for y in range(img_size[0]):
    for x in range(img_size[1]):
        button = Button(description='', layout=Layout(width='30px', height='30px', border='1px solid black'))
        button.style.button_color = 'black' if canvas[y, x] == 0 else 'white'  # Настройка начального цвета кнопок
        button.on_click(lambda b, x=x, y=y: update_pixel(x, y, b))
        grid[y, x] = button

# Функция для отображения текущего состояния canvas и сохранения изображения
def display_canvas_and_save():
    fig, ax = plt.subplots(figsize=(img_size[1], img_size[0]))  # Создаем фигуру с соответствующими размерами
    ax.imshow(canvas, cmap='gray', interpolation='nearest')  # Отображаем текущее состояние canvas
    ax.axis('off')  # Отключаем оси
    fig.tight_layout()
    
    # Сохраняем изображение в PNG формате размером 6x6 пикселей
    fig.savefig('Input.png', dpi=1, transparent=True)  # dpi равен ширине изображения
    plt.close(fig)  # Закрываем фигуру после сохранения
    global test_img_path
    test_img_path = 'Input.png'
    # Показываем сообщение о сохранении
    print('Изображение сохранено как Input.png')

# Кнопка для подтверждения изменений и сохранения изображения
confirm_button = Button(description='Подтвердить и сохранить', button_style='success')
confirm_button.on_click(lambda b: display_canvas_and_save())

# Отображаем сетку кнопок и кнопку подтверждения в вертикальном виджете
display(VBox([grid, confirm_button]))

Изображение сохранено как Input.png
Изображение сохранено как Input.png
Изображение сохранено как Input.png


In [216]:
Test = predict(test_img_path, W)
print("Нарисованный знак вероятно: ", Test)

Нарисованный знак вероятно:  =
